<a href="https://colab.research.google.com/github/bhagirathtallapragada/Secure-AI-project--Implementing-adversarial-attacks-and-defenses-using-ART/blob/main/extraction__attacks_spai_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-gpu==1.15.2 keras==2.2.3 adversarial-robustness-toolbox

     |████████████████████████████████| 410.9 MB 28 kB/s 
     |████████████████████████████████| 312 kB 53.9 MB/s 
     |████████████████████████████████| 1.1 MB 58.2 MB/s 
     |████████████████████████████████| 3.8 MB 34.0 MB/s 
     |████████████████████████████████| 503 kB 76.4 MB/s 
     |████████████████████████████████| 50 kB 7.6 MB/s 
     |████████████████████████████████| 3.3 MB 51.0 MB/s 
     |████████████████████████████████| 26.3 MB 91 kB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=a574b29bfd743df115c589b476d8a7c163ce2122f0bfbb7c43dd3f22680703ca
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2

In [ ]:
!pip install keras==1.2.2

     |████████████████████████████████| 175 kB 35.6 MB/s 
     |████████████████████████████████| 2.8 MB 37.2 MB/s 
  Created wheel for keras: filename=Keras-1.2.2-py3-none-any.whl size=209599 sha256=e8f3a7e01674852c0142582c31c9227bb2a0053eef7948eb855c375dcb722959
  Stored in directory: /root/.cache/pip/wheels/d1/32/23/2a1db3765ec19c91503843380a4f92b6530598949c661c5fa2
  Created wheel for theano: filename=Theano-1.0.5-py3-none-any.whl size=2668111 sha256=912aa70158aefae31daf8bf0bd46bb67153f9cfe8b18c8030d80b9997b05c5a2
  Stored in directory: /root/.cache/pip/wheels/26/68/6f/745330367ce7822fe0cd863712858151f5723a0a5e322cc144
Successfully built keras theano
  Attempting uninstall: keras
    Found existing installation: Keras 2.2.3
    Uninstalling Keras-2.2.3:
      Successfully uninstalled Keras-2.2.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.7

In [ ]:
!pip install adversarial-robustness-toolbox

In [ ]:
from google.colab import drive
drive.mount('/content/drive') #, force_remount = True)

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np
from tensorflow.keras import utils as np_utils
import tensorflow.keras.backend as K
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
# from keras.optimizers import Adam
from tensorflow.keras.layers import InputLayer, Reshape
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import CategoricalCrossentropy


from art.estimators.classification import KerasClassifier

In [ ]:
# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

Untaring file...


In [ ]:
# checking the format of data
print(type(x_train[0]))
print(x_train[0].shape)

# retaining the original format for the test data
x_test_retained=x_test.copy()

# Convert class vectors to binary class matrices.
y_test = tf.keras.utils.to_categorical(y_test, 10)
y_train = tf.keras.utils.to_categorical(y_train, 10)

print(y_test.shape)
print(y_train.shape)

# Normalizing the data by converting to float and dividing by 255 (max value for r,g,b)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train  /= 255
x_test /= 255

# print(x_train[0])

<class 'numpy.ndarray'>
(32, 32, 3)
(10000, 10)
(50000, 10)


In [ ]:
def train_percentage(n):
   
    num_imgs = x_train.shape[ 0 ]
    num_poisoned_images = n
    print( 'Percentage of access to train images', num_poisoned_images*100 / num_imgs )
    idx = np.arange( num_imgs, dtype=np.int )
    np.random.shuffle( idx )
    candidate_images = np.copy( x_train[ idx[ :num_poisoned_images ] ] )
    target_labels=np.copy( y_train[ idx[ :num_poisoned_images ] ] )
    return candidate_images,target_labels

In [ ]:
def create_model():
    model = tf.keras.Sequential( )
    model.add( Conv2D( 32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(32,32,3) ) )
    model.add( Conv2D( 64, (3, 3), padding='same', activation='relu' ) )
    model.add( MaxPooling2D( pool_size=(2, 2) ) )
    model.add( Flatten( ) )
    model.add( Dense( 128, activation='relu' ) )
    model.add( Dense(10, activation='softmax' ) )
    
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                  optimizer='adam',
                  metrics=[ 'accuracy' ] )
    return model

In [ ]:
model_1 = create_model()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
artclassifier2 = KerasClassifier(model=model_1, clip_values=(0, 1), use_logits=False)

## Copycat CNN

### implementing with 5% data access

In [ ]:
from art.attacks.extraction import CopycatCNN
cc_attack = CopycatCNN(classifier = artclassifier2, batch_size_fit=5, 
                        batch_size_query=10,
                        nb_epochs=5,
                        nb_stolen=100)

In [ ]:
x_extr,y_extr = train_percentage(2500)

Percentage of access to train images 5.0


In [ ]:
stolen_classifier = cc_attack.extract(x_extr,y_extr,thieved_classifier=artclassifier2)

Train on 100 samples
Epoch 1/5
100/100 [==============================] - 1s 11ms/sample - loss: 1.6602 - acc: 0.3500
Epoch 2/5
100/100 [==============================] - 1s 9ms/sample - loss: 1.3026 - acc: 0.4600
Epoch 3/5
100/100 [==============================] - 1s 9ms/sample - loss: 1.0285 - acc: 0.5900
Epoch 4/5
100/100 [==============================] - 1s 9ms/sample - loss: 0.7760 - acc: 0.7400
Epoch 5/5
100/100 [==============================] - 1s 9ms/sample - loss: 0.4722 - acc: 0.8500


In [ ]:
stolen_classifier._model.evaluate(x_test,y_test)

10000/10000 [==============================] - 12s 1ms/sample - loss: 8.8520 - acc: 0.0952


[8.851990235900878, 0.0952]

### Implementing with 10% data access

In [ ]:
from art.attacks.extraction import CopycatCNN
cc_attack = CopycatCNN(classifier = artclassifier2, batch_size_fit=5, 
                        batch_size_query=10,
                        nb_epochs=5,
                        nb_stolen=100)

In [ ]:
x_extr,y_extr = train_percentage(5000)

Percentage of access to train images 10.0


In [ ]:
stolen_classifier = cc_attack.extract(x_extr,y_extr,thieved_classifier=artclassifier2)

Train on 100 samples
Epoch 1/5
100/100 [==============================] - 1s 9ms/sample - loss: 0.7882 - acc: 0.6200
Epoch 2/5
100/100 [==============================] - 1s 9ms/sample - loss: 0.4405 - acc: 0.8600
Epoch 3/5
100/100 [==============================] - 1s 9ms/sample - loss: 0.1358 - acc: 0.9700
Epoch 4/5
100/100 [==============================] - 1s 9ms/sample - loss: 0.1160 - acc: 0.9700
Epoch 5/5
100/100 [==============================] - 1s 10ms/sample - loss: 0.0754 - acc: 1.0000


In [ ]:
stolen_classifier._model.evaluate(x_test,y_test)

10000/10000 [==============================] - 12s 1ms/sample - loss: 13.6551 - acc: 0.0943


[13.655105297851563, 0.0943]

### Implementing with 30% data access

In [ ]:
from art.attacks.extraction import CopycatCNN
cc_attack = CopycatCNN(classifier = artclassifier2, batch_size_fit=5, 
                        batch_size_query=10,
                        nb_epochs=5,
                        nb_stolen=100)

In [ ]:
x_extr,y_extr = train_percentage(15000)

Percentage of access to train images 30.0


In [ ]:
stolen_classifier = cc_attack.extract(x_extr,y_extr,thieved_classifier=artclassifier2)

Train on 100 samples
Epoch 1/5
100/100 [==============================] - 1s 10ms/sample - loss: 0.4845 - acc: 0.7800
Epoch 2/5
100/100 [==============================] - 1s 9ms/sample - loss: 0.1372 - acc: 0.9600
Epoch 3/5
100/100 [==============================] - 1s 9ms/sample - loss: 0.0802 - acc: 0.9700
Epoch 4/5
100/100 [==============================] - 1s 9ms/sample - loss: 0.0393 - acc: 0.9900
Epoch 5/5
100/100 [==============================] - 1s 9ms/sample - loss: 0.0174 - acc: 1.0000


In [ ]:
stolen_classifier._model.evaluate(x_test,y_test)

10000/10000 [==============================] - 12s 1ms/sample - loss: 15.3067 - acc: 0.0905


[15.30673016204834, 0.0905]

## Implementing Knockoff Nets

### Implementing with 5% data access

In [ ]:
from art.attacks.extraction import KnockoffNets
ko_attack = KnockoffNets(classifier = artclassifier2, batch_size_fit=5, 
                        batch_size_query=10,
                        nb_epochs=5,
                        nb_stolen=100,reward="all")

In [ ]:
x_extr,y_extr = train_percentage(2500)

Percentage of access to train images 5.0


In [ ]:
stolen_classifier = ko_attack.extract(x_extr,y_extr,thieved_classifier=artclassifier2)

In [ ]:
stolen_classifier._model.evaluate(x_test,y_test)

10000/10000 [==============================] - 12s 1ms/sample - loss: 15.9493 - acc: 0.0981


[15.949310882568358, 0.0981]

### Implementing with 10 % data access

In [ ]:
from art.attacks.extraction import KnockoffNets
ko_attack = KnockoffNets(classifier = artclassifier2, batch_size_fit=5, 
                        batch_size_query=10,
                        nb_epochs=5,
                        nb_stolen=100,reward="all")

In [ ]:
x_exc,y_exc = train_percentage(5000)

Percentage of access to train images 10.0


In [ ]:
stolen_classifier = ko_attack.extract(x_extr,y_extr,thieved_classifier=artclassifier2)

In [ ]:
stolen_classifier._model.evaluate(x_test,y_test)

10000/10000 [==============================] - 11s 1ms/sample - loss: 17.9072 - acc: 0.0969


[17.90721037597656, 0.0969]

### Implementing with 30 % data access

In [ ]:
from art.attacks.extraction import KnockoffNets
ko_attack = KnockoffNets(classifier = artclassifier2, batch_size_fit=5, 
                        batch_size_query=10,
                        nb_epochs=5,
                        nb_stolen=100,reward="all")

In [ ]:
x_extr, y_extr = train_percentage(15000)

Percentage of access to train images 30.0


In [ ]:
stolen_classifier = ko_attack.extract(x_extr,y_extr,thieved_classifier=artclassifier2)

In [ ]:
stolen_classifier._model.evaluate(x_test,y_test)

10000/10000 [==============================] - 11s 1ms/sample - loss: 19.4077 - acc: 0.0951


[19.40772201843262, 0.0951]

## Implementing Functionally Equivalent Extraction

### Implementing with 5% data access

In [ ]:
from art.attacks.extraction import FunctionallyEquivalentExtraction
fe_attack = FunctionallyEquivalentExtraction(classifier = artclassifier2,
                                          num_neurons = 128)

In [ ]:
x_extr,y_extr = train_percentage(2500)

Percentage of access to train images 5.0


In [ ]:
stolen_classifier = fe_attack.extract(x_extr,y_extr,delta_0 = 1, fraction_true = 0.0003, 
                            rel_diff_slope = 1e-05, rel_diff_value = 1e-04, 
                            delta_init_value = 0.1, delta_value_max = 50, 
                            d2_min = 1, d_step = 0.1, delta_sign = 0.02, 
                            unit_vector_scale = 10, ftol = 0)

/usr/local/lib/python3.7/dist-packages/art/attacks/extraction/functionally_equivalent_extraction.py:221: RuntimeWarning: invalid value encountered in true_divide
  if np.sum(np.abs((m_1 - m_2) / m_1) < rel_diff_slope) > fraction_true * self.num_classes:
/usr/local/lib/python3.7/dist-packages/art/attacks/extraction/functionally_equivalent_extraction.py:225: RuntimeWarning: invalid value encountered in true_divide
  t_hat = t_1 + np.divide(y_2 - y_1 - (t_2 - t_1) * m_2, m_1 - m_2)
/usr/local/lib/python3.7/dist-packages/art/attacks/extraction/functionally_equivalent_extraction.py:226: RuntimeWarning: invalid value encountered in true_divide
  y_hat = y_1 + m_1 * np.divide(y_2 - y_1 - (t_2 - t_1) * m_2, m_1 - m_2)
/usr/local/lib/python3.7/dist-packages/art/attacks/extraction/functionally_equivalent_extraction.py:215: RuntimeWarning: invalid value encountered in true_divide
  m_1 = (self._o_l(x_1_p) - self._o_l(x_1)) / epsilon
/usr/local/lib/python3.7/dist-packages/art/attacks/extraction/fu

In [ ]:
stolen_classifier._model.evaluate(x_test,y_test)

### Implementing with 10 % data access

In [ ]:
from art.attacks.extraction import CopycatCNN
fe_attack = FunctionallyEquivalentExtraction(classifier = artclassifier2,
                                          num_neurons = 128)

In [ ]:
x_extr,y_extr = train_percentage(5000)

Percentage of access to train images 10.0


In [ ]:
stolen_classifier = fe_attack.extract(x_extr,y_extr,thieved_classifier=artclassifier2)

Train on 100 samples
Epoch 1/5
100/100 [==============================] - 1s 10ms/sample - loss: 2.1055 - acc: 0.2900
Epoch 2/5
100/100 [==============================] - 1s 10ms/sample - loss: 1.3554 - acc: 0.6200
Epoch 3/5
100/100 [==============================] - 1s 10ms/sample - loss: 0.8453 - acc: 0.7400
Epoch 4/5
100/100 [==============================] - 1s 10ms/sample - loss: 0.3901 - acc: 0.9000
Epoch 5/5
100/100 [==============================] - 1s 10ms/sample - loss: 0.2001 - acc: 0.9800


In [ ]:
stolen_classifier._model.evaluate(x_test,y_test)

10000/10000 [==============================] - 13s 1ms/sample - loss: 3.1154 - acc: 0.2806


[3.1154094215393067, 0.2806]

### Implementing with 30% data access

In [ ]:
from art.attacks.extraction import CopycatCNN
fe_attack = FunctionallyEquivalentExtraction(classifier = artclassifier2,
                                          num_neurons = 128)

In [ ]:
x_extr,y_extr = train_percentage(15000)

Percentage of access to train images 30.0


In [ ]:
stolen_classifier = fe_attack.extract(x_extr,y_extr,thieved_classifier=artclassifier2)

Train on 100 samples
Epoch 1/5
100/100 [==============================] - 1s 10ms/sample - loss: 2.6147 - acc: 0.3200
Epoch 2/5
100/100 [==============================] - 1s 10ms/sample - loss: 1.6351 - acc: 0.5000
Epoch 3/5
100/100 [==============================] - 1s 10ms/sample - loss: 1.0177 - acc: 0.6800
Epoch 4/5
100/100 [==============================] - 1s 10ms/sample - loss: 0.5214 - acc: 0.8400
Epoch 5/5
100/100 [==============================] - 1s 10ms/sample - loss: 0.2608 - acc: 0.9500


In [ ]:
stolen_classifier._model.evaluate(x_test,y_test)

10000/10000 [==============================] - 13s 1ms/sample - loss: 2.7706 - acc: 0.3170


[2.7706076084136964, 0.317]